DC, for the Employment data analysis Please do the Data preparatin as following:
 
Get the following fields from the CICdata
 
customer_id, TIN, run_date, occupationdesc, occupationstatusdesc, TradeName, employment_duration_in_days (DateHiredTo or run_date - DateHiredFrom), PSICdesc, AnnualMonthlyIndicator, Currency, GrossIncome, max_sanctioned_amt_24m, max_creditcard_limit_24m, max_financed_amt_24m, tot_granted_contracts_cnt_24m, tot_granted_contracts_amt_12m, max_sanctioned_amt_12m, Max_CreditCard_limit_12m, max_financed_amt_12m, tot_granted_contracts_cnt_12m, tot_granted_contracts_amt_12m
 
Get the self declared employment details, application_date,  and MaxSalary information from our internal Salary scaling table using the CustomerID or TIN and application_date = run_date
 
Get the gender, age, region, province of the customer from demographic table using customer_id or TIN and application_date = run_date
 
Get the following fields from the Credolab tables using the customer_id or mobile number and application_date = run date:
Device Price
Device Manufacturer (Apple/ Samsung/ Oppo etc.)
Device Model
Device Age (usually derived by the application_date - earliest apps installation date)
number of job search apps installed (linkedin, jobstreet, glint etc.)
number of professional apps installed (office, gdrive etc)
number of ecommerce apps installed (lazada, shopee, shein, zalora, temu etc.)
number of ride hailing apps installed (grab, moveit etc.)
number of finance apps installed (Banking apps, FinTech apps, eWallet apps)
number of Risky apps (gambling apps, crypto apps, payday loan apps)
number of OTT apps (Netflix, PrimeVideo etc.)
Sowndarya: may be you can share your earlier apps categorisation work with DC for this. To keep things simple, just give him your final app categorisation mapping.

# <div align="center" style="color: #C64242;">**Employement Data with CredoLab Data For Income Estimation**</div>

## <div style="color: #D9534F;"> Declaring the library </div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import timeit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
import re
from fuzzywuzzy import process
# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

c:\Users\DwaipayanChakroborti\Myenv\reporting\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## <div style="color: #C64242;"> Combining Granted and Non Granted Contract tables </div>

In [2]:
import time

start_time = time.time()

In [3]:
# combining both granted and non granted tables of CIC

sq = """
WITH
  CICBaseTable AS ( -- Query FOR dfgranted
  SELECT
    digitalLoanAccountId,
    crifApplicationId,
    customerId,
    processEngineGuid,
    requestGuid,
    ContractHistoryType,
    CBContractCode,
    ContractEndDate,
    ContractPhase,
    ContractPhaseDesc,
    ContractStartDate,
    ContractStatus,
    ContractStatusDesc,
    ContractType,
    ContractTypeDesc,
    Currency,
    CurrencyDesc,
    LastUpdateDate,
    OriginalCurrency,
    OriginalCurrencyDesc,
    ProviderCodeEncrypted,
    ProviderContractNo,
    ReferenceNo,
    Role,
    RoleDesc,
    BilledAmount,
    BoardResolutionFlag,
    BoardResolutionFlagDesc,
    CancellationDate,
    CardReferenceCode,
    ChargedAmount,
    CreditLimit,
    CreditPurpose,
    CreditPurposeDesc,
    FinancedAmount,
    FirstPaymentDate,
    FlagCardUsed,
    HolderLiability,
    HolderLiabilityDesc,
    InstallmentType,
    InstallmentTypeDesc,
    InstallmentsNumber,
    LastChargeDate,
    LastPaymentAmount,
    LastPaymentDate,
    MinPaymentIndicator,
    MinPaymentIndicatorDesc,
    MinPaymentPercentage,
    MonthlyPaymentAmount,
    NextPayment,
    NextPaymentDate,
    OutstandingBalance,
    OutstandingBalanceUnbilled,
    OutstandingPaymentsNumber,
    OverallCreditLimit,
    OverdueDays,
    OverdueDaysDesc,
    OverduePaymentsAmount,
    OverduePaymentsNumber,
    PaymentMethod,
    PaymentMethodDesc,
    PaymentPeriodicity,
    PaymentPeriodicityDesc,
    PremiumCard,
    PremiumCardDesc,
    ReorganizedCreditCode,
    ReorganizedCreditCodeDesc,
    ServicesLinesNo,
    TimesCardUsed,
    TransactionType,
    TransactionTypeDesc,
    Utilization,
    LinkedSubject_CBSubjectCode,
    LinkedSubject_Name,
    LinkedSubject_Role,
    LinkedSubject_RoleDesc,
    Note_TypeDesc,
    Note_Text,
    Note_Type,
    run_date,
    NULL AS ContractRequestDate,
    'granted' AS SOURCE
  FROM
    prj-prod-dataplatform.risk_credit_cic_data.granted_contracts
  UNION ALL
    -- Query FOR dfnongranted
  SELECT
    digitalLoanAccountId,
    crifApplicationId,
    customerId,
    processEngineGuid,
    requestGuid,
    NULL AS ContractHistoryType,
    CBContractCode,
    NULL AS ContractEndDate,
    ContractPhase,
    ContractPhaseDesc,
    NULL AS ContractStartDate,
    NULL AS ContractStatus,
    NULL AS ContractStatusDesc,
    ContractType,
    ContractTypeDesc,
    NULL AS Currency,
    NULL AS CurrencyDesc,
    LastUpdateDate,
    NULL AS OriginalCurrency,
    NULL AS OriginalCurrencyDesc,
    ProviderCodeEncrypted,
    ProviderContractNo,
    ReferenceNo,
    Role,
    RoleDesc,
    NULL AS BilledAmount,
    NULL AS BoardResolutionFlag,
    NULL AS BoardResolutionFlagDesc,
    NULL AS CancellationDate,
    NULL AS CardReferenceCode,
    NULL AS ChargedAmount,
    CreditLimit,
    NULL AS CreditPurpose,
    NULL AS CreditPurposeDesc,
    FinancedAmount,
    NULL AS FirstPaymentDate,
    NULL AS FlagCardUsed,
    NULL AS HolderLiability,
    NULL AS HolderLiabilityDesc,
    NULL AS InstallmentType,
    NULL AS InstallmentTypeDesc,
    InstallmentsNumber,
    NULL AS LastChargeDate,
    NULL AS LastPaymentAmount,
    NULL AS LastPaymentDate,
    NULL AS MinPaymentIndicator,
    NULL AS MinPaymentIndicatorDesc,
    NULL AS MinPaymentPercentage,
    MonthlyPaymentAmount,
    NULL AS NextPayment,
    NULL AS NextPaymentDate,
    NULL AS OutstandingBalance,
    NULL AS OutstandingBalanceUnbilled,
    NULL AS OutstandingPaymentsNumber,
    NULL AS OverallCreditLimit,
    NULL AS OverdueDays,
    NULL AS OverdueDaysDesc,
    NULL AS OverduePaymentsAmount,
    NULL AS OverduePaymentsNumber,
    NULL AS PaymentMethod,
    NULL AS PaymentMethodDesc,
    PaymentPeriodicity,
    PaymentPeriodicityDesc,
    NULL AS PremiumCard,
    NULL AS PremiumCardDesc,
    NULL AS ReorganizedCreditCode,
    NULL AS ReorganizedCreditCodeDesc,
    NULL AS ServicesLinesNo,
    NULL AS TimesCardUsed,
    NULL AS TransactionType,
    NULL AS TransactionTypeDesc,
    NULL AS Utilization,
    LinkedSubject_CBSubjectCode,
    LinkedSubject_Name,
    LinkedSubject_Role,
    LinkedSubject_RoleDesc,
    Note_TypeDesc,
    Note_Text,
    Note_Type,
    run_date,
    ContractRequestDate,
    'nongranted' AS SOURCE
  FROM
    prj-prod-dataplatform.risk_credit_cic_data.notgranted_contracts )
select distinct * from CICBaseTable;
"""

dfrawdata = client.query(sq).to_dataframe(progress_bar_type='tqdm')
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

Job ID a9b70293-e7b0-40bc-8384-8cb990cc80df successfully executed: 100%|██████████|
Downloading: 100%|██████████|
Execution time: 1431.1571357250214 seconds


In [4]:
duplicate_rows = dfrawdata[dfrawdata.duplicated()]
len(duplicate_rows)

0

In [5]:
print(f"The row and columns after combining granted and non granted and getting only unique rows are:\t {dfrawdata.shape}")

The row and columns after combining granted and non granted and getting only unique rows are:	 (1593226, 82)


In [6]:
dfrawdata.to_csv(r'C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\IncomeEstimation\Data\cicgrantednongranted.csv', index = False)

In [7]:
%%time

# PSIC description which is a unique code given to job types
sq = """
select PSIC, PSICDesc, AnnualMonthlyIndicator,  max(GrossIncome) maxgrossincome FROM  prj-prod-dataplatform.risk_credit_cic_data.employment_data group by 1, 2, 3;
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 71559088-65a7-4928-8fd6-33cfed522e3a successfully executed: 100%|██████████|
Downloading: 100%|██████████|
CPU times: total: 62.5 ms
Wall time: 4.48 s


In [8]:
# Save the PSIC description in a csv file
dfd.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\IncomeEstimation\Data\PSIC_Description.csv")

In [9]:
def clean_psic_desc(text):
    if pd.isna(text):
        return text
    text = str(text)  # Convert to string
    return re.sub(r'^\d+\s*-\s*', '', text)

In [10]:
# Apply the function to create a new column
dfd['CleanedPSICDesc'] = dfd['PSICDesc'].apply(clean_psic_desc)

# Display the first few rows to verify the result
dfd.head()

,PSIC,PSICDesc,AnnualMonthlyIndicator,maxgrossincome,CleanedPSICDesc
0,55109,55109 - Other short term accommodation activit...,None,None,"Other short term accommodation activities, n.e.c"
1,None,None,Y,997000,None
2,55101,55101 - Hotels and motels,M,70000,Hotels and motels
3,85590,85590 - Other education n.e.c.,None,None,Other education n.e.c.
4,99019,99019 - International organizations and extra-...,None,None,International organizations and extra-territor...


In [11]:
# saving the above details to a csv file

dfd.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\IncomeEstimation\Data\PSICDesc_desc.csv", index = False)

In [12]:
%%time

# Get the data from loans occupation table

sq = """select * from prj-prod-dataplatform.dl_loans_db_raw.tdbk_loans_occupation_mtb;"""

dfoccupation = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 7c22d4e1-04ac-4eba-8ec0-dc1ce4ee5a1a successfully executed: 100%|██████████|
Downloading: 100%|██████████|
CPU times: total: 46.9 ms
Wall time: 900 ms


In [13]:
print(f"The rows and columns in tdbk_loans_occupation_mtb table are:\t {dfoccupation.shape}")

The rows and columns in tdbk_loans_occupation_mtb table are:	 (197, 7)


In [14]:
dfoccupation.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\IncomeEstimation\Data\occupation.csv", index = False)

In [15]:
dfoccupation.head()

,id,name,createdBy,status,createdDt,updatedBy,updatedDt
0,1,Market Gardeners and Crop Growers,IT,true,2022-12-06 17:02:20+00:00,IT,2022-12-06 17:02:20+00:00
1,68,Metal Working Machine Tool Setters and Operators,IT,true,2022-12-06 17:02:24+00:00,IT,2022-12-06 17:02:24+00:00
2,59,"Glass Makers, Cutters, Grinders and Finishers",IT,true,2022-12-06 17:02:24+00:00,IT,2022-12-06 17:02:24+00:00
3,73,Electrical Engineers,IT,true,2022-12-06 17:02:24+00:00,IT,2022-12-06 17:02:24+00:00
4,61,Glass And Ceramics Plant Operators,IT,true,2022-12-06 17:02:24+00:00,IT,2022-12-06 17:02:24+00:00


## <div style="color: #C64242; font-family: Monotype Corsiva;"><b> Main Query Part </b> </div>

In [16]:
%%time

# getting the customer demographic information from loan master table and employment information from cic employment data table

sq ="""# Loan Master 
with loanmaster as --- data from loan master for customer demographic details
(select distinct customerId, digitalLoanAccountId, loanAccountNumber, startApplyDateTime, natureofwork,industryDescription, incomeMultiple, employmentstatus,credolabScore,city,
       barangay, province, postalCode, age, monthlyIncome,  tinNumber, gender, dateOfBirth, email, mobileNo,firstName, middleName, lastName, custLoanJourneyZipCode,
       custLoanJourneyProvince, custLoanJourneyCity,  tsa_onboarding_time,new_loan_type
 from prj-prod-dataplatform.risk_credit_mis.loan_master_table),
employmentdata as ---- data from employment table for employment related information
(SELECT distinct 
  digitalLoanAccountId,
  crifApplicationId,
  customerId,
  coalesce(AnnualMonthlyIndicator, 'NA') AnnualMonthlyIndicator,
  coalesce(DateHiredFrom, format_date('%Y-%m-%d', current_date())) DateHiredFrom,
  DateHiredTo,
  GrossIncome,
  Occupation,
  OccupationDesc,
  OccupationStatus,
  OccupationStatusDesc,
  TIN,
  PhoneNumber,
  PSIC,
  PSICDesc
FROM  prj-prod-dataplatform.risk_credit_cic_data.employment_data)
select  ---joining both the above tables
lm.customerId, lm.digitalLoanAccountId, loanAccountNumber,
       startApplyDateTime, natureofwork, industryDescription,
       incomeMultiple, employmentstatus, credolabScore, city,
       barangay, province, postalCode, age, monthlyIncome,
       tinNumber, gender, dateOfBirth, email, mobileNo, firstName,
       middleName, lastName, custLoanJourneyZipCode,
       custLoanJourneyProvince, custLoanJourneyCity, tsa_onboarding_time,
       new_loan_type, ed.digitalLoanAccountId edprimarykey, crifApplicationId,
       AnnualMonthlyIndicator, DateHiredFrom,
       DateHiredTo, GrossIncome, Occupation, OccupationDesc,
       OccupationStatus, OccupationStatusDesc, TIN, PhoneNumber,
       PSIC, PSICDesc
from loanmaster lm 
left join employmentdata ed on ed.digitalLoanAccountId = lm.digitalLoanAccountId"""

dfdemograph_employment = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID d2862784-f07d-46be-8e67-bc369345615c successfully executed: 100%|██████████|
Downloading: 100%|██████████|
CPU times: total: 3min 10s
Wall time: 20min 23s


In [17]:
print(f"The rows and columns for the combined data from loan master and cic employment table are:\t {dfdemograph_employment.shape}")

The rows and columns for the combined data from loan master and cic employment table are:	 (1591370, 42)


In [18]:
dfdemograph_employment.head()

,customerId,digitalLoanAccountId,loanAccountNumber,startApplyDateTime,natureofwork,industryDescription,incomeMultiple,employmentstatus,credolabScore,city,...,DateHiredTo,GrossIncome,Occupation,OccupationDesc,OccupationStatus,OccupationStatusDesc,TIN,PhoneNumber,PSIC,PSICDesc
0,1999201,94f8ceef-52f3-423d-8e9d-535b8d925aa7,None,2023-12-29 12:44:23,Call Center Agent/Tele Marketer,Services,NaN,SALARIED,489,CITY OF MANDALUYONG,...,None,None,None,None,None,None,None,02-8667-6800,None,None
1,2202798,a0a45f7a-6ccb-411c-89ab-e2072e5cc752,None,2024-01-13 11:50:39,Consultant,Technology,NaN,SALARIED,484,CITY OF MUNTINLUPA,...,None,None,None,None,None,None,None,None,None,None
2,1344082,02f7091a-f4c7-4b7a-adba-20c5524172ee,None,2022-04-25 05:56:29,Admin/Secretarial,Services,NaN,SALARIED,506.0,SANTA ROSA,...,None,None,None,None,None,None,None,None,None,None
3,2664511,e2b3050f-063b-45f8-b4cf-1169d94054b3,None,2024-07-16 13:50:50,None,Manufacturing,NaN,BUSINESS OWNER/SOLE PROPRIETOR,451.0,BINANGONAN,...,None,None,None,None,None,None,None,None,None,None
4,2615579,eac50116-e167-4ef2-a1d4-fcddbe2322b3,None,2024-06-28 10:05:48,None,Manufacturing,NaN,SELF EMPLOYED,395.0,CITY OF PASIG,...,None,None,None,None,None,None,None,None,None,None


In [19]:
dfdemograph_employment['PSICDesc'].value_counts()

PSICDesc
822 - Call centers and other related activities                        1454
479 - Retail trade not in stores, stalls or markets                     573
6209 - Other information technology and computer service activities     506
64199 - Banking activities, n.e.c.                                      465
8560 - Educational support services                                     352
                                                                       ... 
47521 - Retail sale of hardware materials                                 1
01450 - Hog farming                                                       1
50121 - Ocean freight transport                                           1
45301 - Wholesale of motor vehicles parts and accessories                 1
351 - Electric power generation, transmission and distribution            1
Name: count, Length: 222, dtype: int64

In [20]:
# Apply the function to create a new column
dfdemograph_employment['CleanedPSICDesc'] = dfdemograph_employment['PSICDesc'].apply(clean_psic_desc)
dfdemograph_employment.columns

Index(['customerId', 'digitalLoanAccountId', 'loanAccountNumber',
       'startApplyDateTime', 'natureofwork', 'industryDescription',
       'incomeMultiple', 'employmentstatus', 'credolabScore', 'city',
       'barangay', 'province', 'postalCode', 'age', 'monthlyIncome',
       'tinNumber', 'gender', 'dateOfBirth', 'email', 'mobileNo', 'firstName',
       'middleName', 'lastName', 'custLoanJourneyZipCode',
       'custLoanJourneyProvince', 'custLoanJourneyCity', 'tsa_onboarding_time',
       'new_loan_type', 'edprimarykey', 'crifApplicationId',
       'AnnualMonthlyIndicator', 'DateHiredFrom', 'DateHiredTo', 'GrossIncome',
       'Occupation', 'OccupationDesc', 'OccupationStatus',
       'OccupationStatusDesc', 'TIN', 'PhoneNumber', 'PSIC', 'PSICDesc',
       'CleanedPSICDesc'],
      dtype='object')

In [21]:
dfdemograph_employment['CleanedPSICDesc'].unique()

array([None, 'Non-specialized wholesale trade',
       'Public administration, local government',
       'Manufacture of wearing apparel, n.e.c',
       'Banking activities, n.e.c.', 'Other manufacturing, n.e.c.',
       'Other information technology and computer service activities',
       'Call centers and other related activities',
       'Other professional, scientific and technical activities, n.e.c.',
       'Business support service activities, n.e.c.',
       'Other telecommunications service activities, n.e.c.',
       'Undifferentiated services-producing activities of private households for own use',
       'Activities of households as employers of domestic personnel',
       'Restaurants', 'Other personal service activities, n.e.c.',
       'Credit cooperative activities', 'Insurance',
       'Service activities incidental to air transportation',
       'Educational support services',
       'Retail trade not in stores, stalls or markets',
       'Travel agency activities', 

In [ ]:
dfdemograph_employment.drop(columns = 'PSICDesc', inplace = True)
dfdemograph_employment.rename(columns = {'CleanedPSICDesc':'PSICDesc'}, inplace = True)


In [23]:
dfdemograph_employment.columns

Index(['customerId', 'digitalLoanAccountId', 'loanAccountNumber',
       'startApplyDateTime', 'natureofwork', 'industryDescription',
       'incomeMultiple', 'employmentstatus', 'credolabScore', 'city',
       'barangay', 'province', 'postalCode', 'age', 'monthlyIncome',
       'tinNumber', 'gender', 'dateOfBirth', 'email', 'mobileNo', 'firstName',
       'middleName', 'lastName', 'custLoanJourneyZipCode',
       'custLoanJourneyProvince', 'custLoanJourneyCity', 'tsa_onboarding_time',
       'new_loan_type', 'edprimarykey', 'crifApplicationId',
       'AnnualMonthlyIndicator', 'DateHiredFrom', 'DateHiredTo', 'GrossIncome',
       'Occupation', 'OccupationDesc', 'OccupationStatus',
       'OccupationStatusDesc', 'TIN', 'PhoneNumber', 'PSIC', 'PSICDesc'],
      dtype='object')

## CredoLab Data

In [24]:
%%time

# CredoLab Data 

sq = """Select  DISTINCT t1.customerId,t3.crodolabDeviceId, ca.package_name
FROM
`prj-prod-dataplatform.risk_credit_mis.loan_master_table` t1
LEFT JOIN
`prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application` t2
ON t1.digitalLoanAccountId = t2.digitalLoanAccountId
LEFT JOIN
`prj-prod-dataplatform.dl_loans_db_raw.tdbk_credolab_track` t3
ON t2.credolabRefNumber = t3.refno
LEFT JOIN
`prj-prod-dataplatform.credolab_raw.android_credolab_datasets_struct_columns` t4
ON t3.refno = t4.deviceId
inner join
`prj-prod-dataplatform.core_raw.loan_accounts` loan
on loan.CUSTOMERID = t1.customerId
 INNER JOIN
(select deviceId, af.package_name as package_name, af.first_install_time as first_install_time from `prj-prod-dataplatform.credolab_raw.android_credolab_Application`  ,
unnest(Application) as af) ca
ON ca.deviceId = t3.refno;"""

df_risk_table_2  = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
print(f"The rows and columns in the credolab dataframe are:\t {df_risk_table_2.shape}")

Job ID d2e1adeb-52f3-4350-a219-fb71eba192c5 successfully executed: |          |
Downloading: 100%|██████████|
The rows and columns in the credolab dataframe are:	 (30395312, 3)
CPU times: total: 5min 23s
Wall time: 59min 14s


In [25]:
df_risk_table_2.head()

,customerId,crodolabDeviceId,package_name
0,1965864,6012b089eef3a0d2b825a10a3b7d5dd3,com.android.bluetooth
1,2034774,01b87d9ebe79859d594790c12487b6e9,com.transsion.fmradio
2,2286766,c396c402b2aba4e77d9762aa634125b5,com.android.proxyhandler
3,2437205,9accf423f9cf39cd381c7a88b3b472fa,com.android.chrome
4,2336244,787c44bd3f3c172b0b7ccb668359f94b,com.samsung.advp.imssettings


In [27]:
df_risk_table_2.package_name.value_counts().to_csv("check.csv")

In [54]:
dummydf = pd.read_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\IncomeEstimation\dummydf.csv")
dummydf.head()

,category,apps
0,social_media,messanger
1,social_media,facebook
2,social_media,lite
3,social_media,facebooklite
4,social_media,telegram


In [55]:
dummydf.shape

(8298, 2)

In [56]:
import csv

def csv_to_category_app_dict(csv_file):
  """Converts a CSV file with columns "Category" and "App" to a dictionary with categories as keys and app lists as values."""
  data = {"category": [], "apps": []}
  with open(csv_file, 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row
    for row in reader:
      category, app = row
      data["category"].append(category)
      data["apps"].append(app)
  return data

# Example usage:
csv_file = r'C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\IncomeEstimation\dummydf.csv'
data = csv_to_category_app_dict(csv_file)
print(data)

{'category': ['social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'social_media', 'Health & lifestyle', 'Health & lifestyle', 'Health & lifestyle', 'Health & lifestyle', 'Health & lifestyle', 'Health & lifestyle', 'Health & lifestyle', 'Health & lifestyle', 'Health & lifestyle', 'Health & lifestyle', 'Health & lifestyle', 'Health & lifestyle', 'Health & lifestyle', 'Health & lifestyle', 'Health & lifestyle', 'Health & lifestyle', 'Health & lifestyle', 'Health & lifestyle', 'Health & lifestyle', 'ecommerce', 'ecommerce', 'ecommerce', 'ecommerce', 'eco

In [50]:
# Create a dummy dataframe with category and app 

# data =    {"category" :['social_media',"social_media","social_media","social_media",'social_media',"social_media",
#                         "social_media",'social_media',"social_media","social_media","social_media",'social_media',
#                         "social_media","social_media","social_media","social_media",'social_media',"social_media",
#                         "social_media","social_media",'social_media',"social_media","social_media",'social_media',
#                         "social_media",'social_media',"social_media","social_media","social_media",'social_media',
#                         "social_media","social_media",'social_media',"social_media",'social_media',"social_media",
#                         "social_media",'social_media',
#                         "Health & lifestyle",  "Health & lifestyle",  "Health & lifestyle", "Health & lifestyle","Health & lifestyle",
#                         "Health & lifestyle",  "Health & lifestyle","Health & lifestyle",  "Health & lifestyle",  "Health & lifestyle", 
#                        "Health & lifestyle",  "Health & lifestyle",  "Health & lifestyle",  "Health & lifestyle",  "Health & lifestyle",
#                        "Health & lifestyle",  "Health & lifestyle",  "Health & lifestyle",  "Health & lifestyle",
                        
#                         "ecommerce","ecommerce","ecommerce","ecommerce",'ecommerce',"ecommerce","ecommerce","ecommerce","ecommerce",'ecommerce',
#                         "ecommerce","ecommerce","ecommerce","ecommerce",'ecommerce', "ecommerce","ecommerce","ecommerce","ecommerce",'ecommerce',
                        
#                         'productivity','productivity','productivity','productivity', 'productivity',
#                         'productivity','productivity','productivity','productivity','productivity',
#                         'productivity','productivity','productivity','productivity','productivity',
#                         'productivity','productivity','productivity','productivity', 'productivity',
#                         'productivity','productivity','productivity','productivity','productivity',
                        
#                         "travel","travel","travel","travel","travel","travel","travel","travel","travel","travel",
#                         "travel","travel","travel","travel","travel","travel","travel","travel","travel",
#                         "VPN","VPN", "VPN","VPN","VPN","VPN","VPN","VPN","VPN",
                        
#                          "finance","finance","finance","finance","finance","finance","finance","finance","finance","finance",
#                         "finance","finance","finance","finance","finance","finance","finance","finance","finance","finance",
#                        "finance","finance","finance","finance","finance","finance","finance","finance","finance","finance",
                        
#                         "games",'games','games','games','games','games','games','games','games','games',"games",'games',
#                         'games','games','games','games','games','games','games','games',"games",'games','games','games',
#                         'games','games','games','games','games','games','games','games','games','games',
                        
#                         "News","News", "News","News","News","News","News","News","News","News",
#                         "News","News", "News","News","News","News","News","News","News","News","News",
                        
#                         "Podcast", "Podcast", "Podcast", "Podcast", "Podcast", "Podcast", "Podcast", "Podcast","Podcast", "Podcast",
#                          "Podcast", "Podcast", "Podcast", "Podcast", "Podcast", "Podcast", "Podcast", "Podcast","Podcast", "Podcast","Podcast",
#                         "Entertainment","Entertainment","Entertainment","Entertainment","Entertainment","Entertainment","Entertainment","Entertainment",
#                         "Entertainment","Entertainment","Entertainment","Entertainment","Entertainment","Entertainment","Entertainment","Entertainment"

                     
#                        ],
                       
#           "apps" :["messanger","facebook","viber","instagram","lite","facebooklite","Telegram","Discord","Pinterest",
#                    "bilibili","reddit","Twitter","linkedin", 'Instagram', 'Twitter', 'LinkedIn', 'Snapchat',
#                    'TikTok', 'Pinterest', 'WhatsApp', 'Reddit', 'YouTube', 'Tumblr', 'Skype', 
#                    'WeChat', 'LINE', 'Viber', 'Signal', 'Twitch', 'VKontakte', 'Weibo', 'Clubhouse', 
#                    'Flickr', 'Quora', 'Badoo', 'Meetup', 'KakaoTalk', 'VK',
                   
#                    'MyFitnessPal', 'Calm', 'Headspace', 'Fitbit', 'NikeTrainingClub', 'LoseIt', 'Strava', 'Yoga Studio', 
#                    'DailyYoga', 'MapMyRun', '7MinuteWorkout', 'PeriodTracker', 'WaterDrinkReminder',
#                     'SleepCycle', 'GoodRx', 'Zombies', 'Fooducate', 'Sweatcoin', 'Habitica',
                   
#                   'Shopee', 'Lazada', 'Zalora', 'Zilingo', 'Amazon', 'eBay', 'AliExpress', 'Wish', 'Tokopedia', 'JD.com', 
#                    'Walmart', 'Flipkart', 'Alibaba', 'Rakuten', 'Taobao', 'Snapdeal', 'MercadoLibre', 'Daraz', 'Olx', 'Etsy',
                   
#                    'googlecanlender','googledrive','samsungcalender','samsungnotes','openai','chatgpt','googledocs',
#                 'camsoftinformation','googlesheets','wps','notes','camscanner','microsoft','googlesheets','Adobe',
#                 'Alarm','onedrive','Notisave','calculator','notepad','pdfreader','pdf','filemanger','calender','outlook',
                   
#                    'Booking', 'Airbnb', 'Uber', 'Lyft', 'GoogleMaps', 'GoogleEarth', 'TripAdvisor', 'Expedia', 
#                    'KAYAK', 'Skyscanner', 'Agoda', 'Hopper', 'Trivago', 'TripIt', 'Traveloka', 'Priceline', 'Orbitz', 'Waze', 'Rome2rio',
                   
#                    "NordVPN","SurfShark", "Express","cyberghost","IPVanish","Private Internet Access",
#                    "Mullvad","Proton", "Atlas",
                   
#                    "Gcash","BPI","Metrobank","iWant","PayMaya","wallet","mocamoca","moca","wise","trust","mrcash",
#                    "PayMaya","Coins", "BPI","UnionBank", "CIMB", "Security Bank","BDO","Metrobank","Robinsons",
#                     "LANDBANK", "EastWest","PNB", "PSBank", "Maybank2U","Citibank","RCBC", "AXA", "SunLife","AUB",
                   
#                    'mobilelegends', 'freefireth', 'spacemafia', 'clashofclans', 'ig', 'shooter', 'bh3global', 'romg', 
#                    'gp', 'GenshinImpact', 'fortnite', 'n9', 'clashroyale', 'subwaysurf', 'gardenscapes', 'homescapes', 
#                    'candycrushsaga', 'eightballpool', 'mobilehero', 'common', 'legends', 'freefireth', 'spacemafia', 
#                    'clashofclans', 'ig', 'shooter', 'romg', 'GenshinImpact', "game","gamecenter",
#                    "battle","survival","mcpe","tycoon",
                   
#                    'Google News', 'BBC News', 'CNN', 'The New York Times', 'The Guardian', 'Reuters', 'Al Jazeera', 
#                    'Fox News', 'NPR News', 'ABC News', 'NBC News', 'Bloomberg', 'CBS News', 'Yahoo News', 'Washington Post', 
#                    'HuffPost', 'Associated Press', 'BBC Sport', 'Sky News', 'ESPN', "News",
                   
#                    'Spotify', 'Apple Podcasts', 'Google Podcasts', 'Stitcher', 'Podbean', 'TuneIn', 'Pocket Casts', 'Overcast',
#                    'Castbox', 'Deezer', 'iHeartRadio', 'RadioPublic', 'Acast', 'Podcast Addict', 'Player FM', 'Breaker', 'SoundCloud', 
#                    'Anchor', 'Audible', 'Podyssey', 'Podcasts'
                   
#                    "Netflix", "loklok", "Mivideo", "Youtube", "viu", "Niki", "Disney", "shorttv", "Amazonprime", 
#                    "Dramabox", "vivamax", "steam", "Hitv", "HBO", "HDmovies", "smartlivestream"


#                   ]}
                
# dummydf = pd.DataFrame(data)
# dummydf['apps'] = dummydf['apps'].str.lower()
# dummydf = dummydf.drop_duplicates(keep='last')
# len(dummydf)

5050

In [57]:
dummydf.head()

,category,apps
0,social_media,messanger
1,social_media,facebook
2,social_media,lite
3,social_media,facebooklite
4,social_media,telegram


In [58]:
dummydf.shape

(8298, 2)

In [59]:
dummydf.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\IncomeEstimation\Data\dummydf.csv", index = False)

In [46]:
def extract_last_word(text):
    parts = text.split('.') 
    return parts[-1]  

def extract_word_between_dots(text):
  """Extracts the word between the first and second dots in a text string.

  Args:
    text: The input text string.

  Returns:
    The extracted word, or None if not found.
  """

  parts = text.split('.')
  if len(parts) >= 3:
    return parts[1]
  else:
    return None

df_risk_table_2["apps"] = df_risk_table_2["package_name"].apply(extract_last_word)
df_risk_table_2["apps_2"] = df_risk_table_2["package_name"].apply(extract_word_between_dots)

df_risk_table_2.head()

,customerId,crodolabDeviceId,package_name,apps,apps_2
0,1965864,6012b089eef3a0d2b825a10a3b7d5dd3,com.android.bluetooth,bluetooth,android
1,2034774,01b87d9ebe79859d594790c12487b6e9,com.transsion.fmradio,fmradio,transsion
2,2286766,c396c402b2aba4e77d9762aa634125b5,com.android.proxyhandler,proxyhandler,android
3,2437205,9accf423f9cf39cd381c7a88b3b472fa,com.android.chrome,chrome,android
4,2336244,787c44bd3f3c172b0b7ccb668359f94b,com.samsung.advp.imssettings,imssettings,samsung


In [31]:
df_risk_table_2.apps.value_counts(dropna= False)

apps
overlay                 729079
android                 545106
service                 332747
services                266337
settings                242859
                         ...  
playville                    1
lullabies2                   1
afbf76f2681f7a3de_v2         1
MagicUniversity              1
a5b65e4dbb2d61746_v2         1
Name: count, Length: 85484, dtype: int64

In [32]:
df_risk_table_2.apps.value_counts().to_csv("uniqueAppsasperlastwordinpackage.csv")

In [65]:
df_cat_merged = df_risk_table_2.merge(dummydf, on=['apps'], how='left')

MemoryError: Unable to allocate 8.93 GiB for an array with shape (1198726461,) and data type int64

In [60]:
# df_cat_merged = df_risk_table_2.merge(dummydf,on=['apps'], how ='left')

def merge_with_category_fallback(df_risk_table_2, dummydf):
  """Merges two DataFrames and assigns category based on matching or fallback."""
  merged_df = df_risk_table_2.merge(dummydf, on=['apps'], how='left')

  # Assign category based on matching or fallback
  merged_df['category'] = merged_df['category'].where(merged_df['category'].notna(), merged_df['apps_2'])

  return merged_df

# Example usage
df_cat_merged = merge_with_category_fallback(df_risk_table_2.copy(), dummydf.copy())

MemoryError: Unable to allocate 8.93 GiB for an array with shape (1198726461,) and data type int64

In [64]:
def merge_with_category_fallback_chunked(df_risk_table_2, dummydf, chunksize=100000):
  """Merges two DataFrames and assigns category based on matching or fallback, processing in chunks."""
  merged_df = pd.DataFrame()

  for start_idx in range(0, len(df_risk_table_2), chunksize):
      end_idx = min(start_idx + chunksize, len(df_risk_table_2))
      chunk = df_risk_table_2.iloc[start_idx:end_idx].copy()
      merged_chunk = chunk.merge(dummydf, on=['apps'], how='left')
      merged_chunk['category'] = merged_chunk['category'].where(merged_chunk['category'].notna(), merged_chunk['apps_2'])
      merged_df = pd.concat([merged_df, merged_chunk], ignore_index=True)  # Append chunks
  return merged_df

# Example usage
df_cat_merged = merge_with_category_fallback_chunked(df_risk_table_2.copy(), dummydf.copy())

MemoryError: Unable to allocate 1.68 GiB for an array with shape (1, 225498127) and data type object

In [34]:
df_cat_merged.head()

,customerId,crodolabDeviceId,package_name,apps,category
0,1965864,6012b089eef3a0d2b825a10a3b7d5dd3,com.android.bluetooth,bluetooth,NaN
1,2034774,01b87d9ebe79859d594790c12487b6e9,com.transsion.fmradio,fmradio,NaN
2,2286766,c396c402b2aba4e77d9762aa634125b5,com.android.proxyhandler,proxyhandler,NaN
3,2437205,9accf423f9cf39cd381c7a88b3b472fa,com.android.chrome,chrome,NaN
4,2336244,787c44bd3f3c172b0b7ccb668359f94b,com.samsung.advp.imssettings,imssettings,NaN


In [38]:
df_cat_merged['category'].value_counts(dropna=False)

category
NaN                   29537945
games                   325674
social_media            205387
productivity            163397
Entertainment           112140
finance                  78285
travel                   35609
VPN                      35478
Health & lifestyle        4333
ecommerce                 1372
News                       530
Podcast                     55
Name: count, dtype: int64

In [41]:
df_cat_merged[['package_name','apps','category']].value_counts(dropna=False).to_csv("packagenameappandcategory.csv")